In [58]:
import requests
import pandas as pd
import json,os
import requests
import pandas as pd
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
#sharepoint dev site
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

# Token endpoint
url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

# Get token for Microsoft Graph
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'https://graph.microsoft.com/.default'
}

token_response = requests.post(url, data=data)
access_token = token_response.json()['access_token']

veeva_headers = {
    'Authorization': '053C78ACD9A27EA36A32FAAC65F3CE72A78B97F7AA9176ED933313F362FD0024132D77598543F2502C16BF103E3A09D2D35814FABAF972DDA2F6FF59086E5918',  # Add your token
    'Content-Type': 'application/x-www-form-urlencoded',
}
sp_headers = {
        'Authorization': f'Bearer {access_token}',  
        'Content-Type': 'application/octet-stream'
        }
drive_id = 'b!9_dFAypENEaXiuuOFt6lus3kX6cO3U5KvGeckT3yjRGApZ-K82XyQLCxWq_PAK6q'

# LOCAL_DOWNLOAD_PATH = './'

to lists all drives in a site

In [8]:
drives_url = "https://graph.microsoft.com/v1.0/sites/envu.sharepoint.com:/sites/IndiaProjectsToday:/drives"

document_info = []
upload_response  = requests.get(drives_url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))

{
   "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#drives",
   "value": [
      {
         "createdDateTime": "2025-04-05T23:07:34Z",
         "description": "",
         "id": "b!zrKbf_sAU02dgYdl0EDVLC-GVE9ofPVLq6XicSlFQJ6Hunrz0YUpQ6GGoDf0py0L",
         "lastModifiedDateTime": "2025-11-12T07:00:15Z",
         "name": "Documents",
         "webUrl": "https://envu.sharepoint.com/sites/IndiaProjectsToday/Shared%20Documents",
         "driveType": "documentLibrary",
         "createdBy": {
            "user": {
               "displayName": "System Account"
            }
         },
         "lastModifiedBy": {
            "user": {
               "email": "ESSPAdminAccount1@envu.com",
               "id": "dc6a2195-0e01-47d0-9fe6-4b28a6c2adb3",
               "displayName": "SharePoint Admin"
            }
         },
         "owner": {
            "user": {
               "email": "ESSPAdminAccount1@envu.com",
               "id": "dc6a2195-0e01-47d0-9fe6-4b28a6c2adb3"

To lists all folders,files and metadata for  a perticular Drive ID

In [29]:
drive_id = 'b!zrKbf_sAU02dgYdl0EDVLC-GVE9ofPVLq6XicSlFQJ6Hunrz0YUpQ6GGoDf0py0L'
url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/children"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}
upload_response  = requests.get(url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))


{
   "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#Collection(driveItem)",
   "value": [
      {
         "createdBy": {
            "application": {
               "id": "aed3878c-05a8-48ea-9de8-864434e31b23",
               "displayName": "Microsoft Graph"
            },
            "user": {
               "displayName": "SharePoint App"
            }
         },
         "createdDateTime": "2025-11-12T07:00:15Z",
         "eTag": "\"{B038E2B0-224C-4DE1-82EB-048A9D6440BB},1\"",
         "id": "01PDDOVQ5Q4I4LATBC4FGYF2YERKOWIQF3",
         "lastModifiedBy": {
            "application": {
               "id": "aed3878c-05a8-48ea-9de8-864434e31b23",
               "displayName": "Microsoft Graph"
            },
            "user": {
               "displayName": "SharePoint App"
            }
         },
         "lastModifiedDateTime": "2025-11-12T07:00:15Z",
         "name": "Documents",
         "parentReference": {
            "driveType": "documentLibrary",
       

To list all Fields/Metadata for a perticular item_id (File/Folder ID)

In [31]:
item_id = '01PDDOVQ5N6T6K3JUD4FCJEQDWB23KAGJL'
drive_id = 'b!zrKbf_sAU02dgYdl0EDVLC-GVE9ofPVLq6XicSlFQJ6Hunrz0YUpQ6GGoDf0py0L'
fields_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/children"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}
upload_response  = requests.get(fields_url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))

{
   "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#Collection(driveItem)",
   "value": [
      {
         "@microsoft.graph.downloadUrl": "https://envu.sharepoint.com/sites/IndiaProjectsToday/_layouts/15/download.aspx?UniqueId=a54929c8-b388-45d4-9609-19f083679388&Translate=false&tempauth=v1.eyJzaXRlaWQiOiI3ZjliYjJjZS0wMGZiLTRkNTMtOWQ4MS04NzY1ZDA0MGQ1MmMiLCJhcHBfZGlzcGxheW5hbWUiOiJTaGFyZVBvaW50IERldiBzaXRlIiwibmFtZWlkIjoiYWVkMzg3OGMtMDVhOC00OGVhLTlkZTgtODY0NDM0ZTMxYjIzQGM0ZGVkYjc0LWQ5MTYtNGVmNC1iNmI1LWFmODBjNTllOTc0MiIsImF1ZCI6IjAwMDAwMDAzLTAwMDAtMGZmMS1jZTAwLTAwMDAwMDAwMDAwMC9lbnZ1LnNoYXJlcG9pbnQuY29tQGM0ZGVkYjc0LWQ5MTYtNGVmNC1iNmI1LWFmODBjNTllOTc0MiIsImV4cCI6IjE3NjI5MzU4MjYifQ.CkAKDGVudHJhX2NsYWltcxIwQ0tIZjBNZ0dFQUFhRm1ReFVURnNkM000YVVVdFoyRkpORGxqTmtsU1FVRXFBQT09CjIKCmFjdG9yYXBwaWQSJDAwMDAwMDAzLTAwMDAtMDAwMC1jMDAwLTAwMDAwMDAwMDAwMAoKCgRzbmlkEgI2NBILCNLQ97TO8dA-EAUaDTIwLjE5MC4xMzIuNDEqLDhERU1laDBDUjhGVEF3Z1c4eHhCemptK0ZYL0toRHBUQTNNQzArZnNvNE09MIwBOAFCEKHYkf0d8ADgmiKG_

To list all Fields for a perticular item_id (Folder ID)

In [39]:
item_id = '01PDDOVQZX5GZ4WDGZ6VDYW5OW2RQH7R7J'
drive_id = 'b!zrKbf_sAU02dgYdl0EDVLC-GVE9ofPVLq6XicSlFQJ6Hunrz0YUpQ6GGoDf0py0L'
fields_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/listItem/fields"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}
upload_response  = requests.get(fields_url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))

{
   "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#drives('b%21zrKbf_sAU02dgYdl0EDVLC-GVE9ofPVLq6XicSlFQJ6Hunrz0YUpQ6GGoDf0py0L')/items('01PDDOVQZX5GZ4WDGZ6VDYW5OW2RQH7R7J')/listItem/fields/$entity",
   "@odata.etag": "\"cbb3e937-d90c-47f5-8b75-d6d4607fc7e9,2\"",
   "FileLeafRef": "LOS Fludora Fusion IQV.docx",
   "_x006d_d5": "64383",
   "id": "13",
   "ContentType": "Document",
   "Created": "2025-11-12T07:04:51Z",
   "AuthorLookupId": "24",
   "Modified": "2025-11-12T07:04:57Z",
   "EditorLookupId": "24",
   "_CheckinComment": "",
   "LinkFilenameNoMenu": "LOS Fludora Fusion IQV.docx",
   "LinkFilename": "LOS Fludora Fusion IQV.docx",
   "DocIcon": "docx",
   "FileSizeDisplay": "23712",
   "ItemChildCount": "0",
   "FolderChildCount": "0",
   "_ComplianceFlags": "",
   "_ComplianceTag": "",
   "_ComplianceTagWrittenTime": "",
   "_ComplianceTagUserId": "",
   "_CommentCount": "",
   "_LikeCount": "",
   "_DisplayName": "Restricted",
   "Edit": "0",
   "_UIVersionStri

Veeva-D365

In [ ]:
import requests
import pandas as pd
import json

drive_id = 'b!NIqRGomSlkK-sVy_p8O3ds3kX6cO3U5KvGeckT3yjRFiYhc0-ZK6Q6P5lvuiy7R0'
url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/children"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

document_info = []
while url:
    response = requests.get(url, headers=sp_headers).json()
    items = response.get('value', [])
    
    for record in items:
        internal_record = {
            'id': record.get('id'),
            'filename': record.get('name'),
            'createdDateTime': record.get('createdDateTime'),
            'lastModifiedDateTime': record.get('lastModifiedDateTime'),
            'webUrl': record.get('webUrl')
            
        }
        document_info.append(internal_record)
    
    # Check for next page link
    url = response.get('@odata.nextLink')

# Convert to DataFrame
document_df = pd.DataFrame(document_info)
document_df = document_df[['id','webUrl']]

print(f"✅ Total documents fetched: {len(document_df)}")
document_df

In [ ]:
# Your existing document DataFrame with 'id' column
ids = document_df['id'].tolist()
# ids = ids[:10]
drive_id = 'b!zrKbf_sAU02dgYdl0EDVLC-GVE9ofPVLq6XicSlFQJ6Hunrz0YUpQ6GGoDf0py0L'
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'
}

# Function to fetch one file’s metadata
def fetch_fields(item_id):
    fields_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/listItem/fields"
    try:
        response = requests.get(fields_url, headers=sp_headers, timeout=10)
        if response.status_code == 200:
            fields = response.json()
            return {
                'id': item_id,
                'DocId':fields.get('DocId'),
                'SpecificationNumber': fields.get('SpecificationNumber'),
                'filename': fields.get('FileName'),
                'DocumentType': fields.get('DocumentType'),
                'DocumentDescription': fields.get('DocumentDescription'),
                'DocumentStatus': fields.get('DocumentStatus'),
                'Modified':fields.get('Modified'),
                'Created':fields.get('Created')
            }
        else:
            return {'id': item_id, 'error': response.status_code}
    except Exception as e:
        return {'id': item_id, 'error': str(e)}

# Use a thread pool for concurrency
meta_data = []
with ThreadPoolExecutor(max_workers=20) as executor:  # You can tune max_workers (10–30 works well)
    futures = {executor.submit(fetch_fields, i): i for i in ids}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching metadata"):
        meta_data.append(future.result())

# Convert to DataFrame
meta_data_df = pd.DataFrame(meta_data)
print(meta_data_df.head())


Fetching metadata: 100%|██████████| 4889/4889 [04:15<00:00, 19.11it/s]

                                   id   DocId SpecificationNumber  \
0  01IHJB5C44HMPL4LPHSJDLEAZSJEI2KJTZ  179265        102000009433   
1  01IHJB5CZ2AFNB2I425BCJAY6H5ZWRD7FZ  179278        102000009658   
2  01IHJB5C5J63UMNPHF5FCJTL6XJK2CDFVM  179272        102000009407   
3  01IHJB5CZGPZQSA67YF5F26KVCRMDQZ4AW  179273        102000009357   
4  01IHJB5CZPHMI4VM7ATNCYQCBBE4F6ZBHV  179262        102000009440   

          filename    DocumentType DocumentDescription DocumentStatus  \
0  102000009433-02  ZSpec_IBDV_URL     102000009433-02       Approved   
1  102000009658-01  ZSpec_IBDV_URL     102000009658-01       Approved   
2  102000009407-02  ZSpec_IBDV_URL     102000009407-02       Approved   
3  102000009357-02  ZSpec_IBDV_URL     102000009357-02       Approved   
4  102000009440-01  ZSpec_IBDV_URL     102000009440-01       Approved   

               Modified               Created  
0  2025-11-05T11:58:37Z  2025-10-17T10:21:10Z  
1  2025-11-05T11:58:37Z  2025-10-17T10:21:11Z  
2 

In [8]:
meta_data_df

,id,DocId,SpecificationNumber,filename,DocumentType,DocumentDescription,DocumentStatus,Modified,Created
0,01IHJB5C44HMPL4LPHSJDLEAZSJEI2KJTZ,179265,102000009433,102000009433-02,ZSpec_IBDV_URL,102000009433-02,Approved,2025-11-05T11:58:37Z,2025-10-17T10:21:10Z
1,01IHJB5CZ2AFNB2I425BCJAY6H5ZWRD7FZ,179278,102000009658,102000009658-01,ZSpec_IBDV_URL,102000009658-01,Approved,2025-11-05T11:58:37Z,2025-10-17T10:21:11Z
2,01IHJB5C5J63UMNPHF5FCJTL6XJK2CDFVM,179272,102000009407,102000009407-02,ZSpec_IBDV_URL,102000009407-02,Approved,2025-11-05T11:58:37Z,2025-10-17T10:21:11Z
3,01IHJB5CZGPZQSA67YF5F26KVCRMDQZ4AW,179273,102000009357,102000009357-02,ZSpec_IBDV_URL,102000009357-02,Approved,2025-11-05T11:58:37Z,2025-10-17T10:21:11Z
4,01IHJB5CZPHMI4VM7ATNCYQCBBE4F6ZBHV,179262,102000009440,102000009440-01,ZSpec_IBDV_URL,102000009440-01,Approved,2025-11-05T11:58:36Z,2025-10-17T10:21:10Z
...,...,...,...,...,...,...,...,...,...
4884,01IHJB5C34FBLZNYXFL5DLMLM3WBYBNIJP,198642,107000000793,107000000793-02,ZSpec_SeL_URL,107000000793-02,Approved,2025-11-05T12:22:10Z,2025-10-17T10:26:08Z
4885,01IHJB5C3U43BICW5JYZC2AMO6XQXHDTNY,176695,102D00000493,102D00000493-01,ZSpec_IBDV_URL,102D00000493-01,Approved,2025-11-05T12:22:08Z,2025-10-17T10:06:57Z
4886,01IHJB5C5DWFX4CYCFBJHKCBTJ2ENH7DYY,176694,102D00000502,102D00000502-01,ZSpec_IBDV_URL,102D00000502-01,Approved,2025-11-05T12:22:08Z,2025-10-17T10:06:57Z
4887,01IHJB5C4XNDHAQKSBMNG2SPBXZ5S5FJFA,176696,102D00000501,102D00000501-01,ZSpec_IBDV_URL,102D00000501-01,Approved,2025-11-05T12:22:09Z,2025-10-17T10:06:58Z


In [9]:
merge_df = pd.merge(document_df,meta_data_df,on='id',how='left')

In [ ]:
merge_df = merge_df[['DocId','SpecificationNumber','filename','DocumentType','DocumentDescription','DocumentStatus','webUrl','Modified','Created']]
merge_df.rename(columns={'filename': 'FileName', 'webUrl': 'Document URL'}, inplace=True)
# merge_df.to_excel('SharepointLinkUpdateUAT.xlsx',index=False)

In [12]:
merge_df['Modified'] = pd.to_datetime(merge_df['Modified'], utc=True)
merge_df['Created'] = pd.to_datetime(merge_df['Created'], utc=True)

In [14]:
merge_df.to_csv('sharepointProdlinks.csv',index=False)


In [42]:
duplicates = merge_df[merge_df['DocId'].duplicated(keep=False)]
# duplicates.to_csv("duplicates.csv",index=False)
duplicates

,DocId,SpecificationNumber,FileName,DocumentType,DocumentDescription,DocumentStatus,Document URL
332,180179,102000016236,102000016236-01,ZSpec_IBDV_URL,102000016236-01,Obsolete,https://envu.sharepoint.com/sites/EnvuSpecific...
1215,177560,102000056797,102000056797-01,ZSpec_IBDV_URL,102000056797-01,Obsolete,https://envu.sharepoint.com/sites/EnvuSpecific...
1425,177700,102000024934,102000024934-02,Spec_URL,102000024934-02,Obsolete,https://envu.sharepoint.com/sites/EnvuSpecific...
1427,177702,102000024689,102000024689-03,Spec_URL,102000024689-03,Obsolete,https://envu.sharepoint.com/sites/EnvuSpecific...
1433,177708,102000024250,102000024250-01,Spec_URL,102000024250-01,Obsolete,https://envu.sharepoint.com/sites/EnvuSpecific...
...,...,...,...,...,...,...,...
4752,199671,102D00000494,102D00000494-01.02,Spec_URL,102D00000494-01.02,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
4754,197346,102000029699,102000029699-02.05,Spec_URL,102000029699-02.05,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
4757,197346,102000029699,102000029699-02.05,Spec_URL,102000029699-02.05,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
4782,175376,102D00000609,102D00000609-01,ZSpec_SeL_URL,102D00000609-01,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...


In [43]:
duplicates['DocId'].unique()

array(['180179', '177560', '177700', '177702', '177708', '177722',
       '177723', '177755', '178133', '178154', '178207', '178224',
       '178260', '178272', '178297', '178298', '178318', '178352',
       '178354', '178356', '178358', '178611', '178628', '178703',
       '180264', '178746', '178778', '178817', '178878', '178933',
       '175337', '203273', '203275', '203324', '203323', '175376',
       '175394', '199671', '197346'], dtype=object)

In [44]:
merge_df.to_csv('sharepointProdlinks.csv',index=False)

In [45]:
merge_df

,DocId,SpecificationNumber,FileName,DocumentType,DocumentDescription,DocumentStatus,Document URL
0,179262,102000009440,102000009440-01,ZSpec_IBDV_URL,102000009440-01,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
1,179263,102000009442,102000009442-01,ZSpec_IBDV_URL,102000009442-01,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
2,179264,102000009440,102000009440-01,ZSpec_IBDV_URL,102000009440-01,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
3,179265,102000009433,102000009433-02,ZSpec_IBDV_URL,102000009433-02,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
4,179266,102000009442,102000009442-01,ZSpec_IBDV_URL,102000009442-01,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
...,...,...,...,...,...,...,...
4888,199529,107000000532,107000000532-04,ZSpec_SeL_URL,107000000532-04,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
4889,176694,102D00000502,102D00000502-01,ZSpec_IBDV_URL,102D00000502-01,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
4890,176695,102D00000493,102D00000493-01,ZSpec_IBDV_URL,102D00000493-01,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
4891,180199,102D00000221,102D00000221-01,ZSpec_IBDV_URL,102D00000221-01,Approved,https://envu.sharepoint.com/sites/EnvuSpecific...
